In [1]:
!pip install gensim

In [2]:
!pip install tqdm

In [3]:
import gensim 
from gensim.models import Word2Vec , KeyedVectors

In [4]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')
vec_king = wv['king']

In [5]:
import pandas as pd 

messages = pd.read_csv('SMSSpamCollection.txt',sep='\t',names=['labels','message'])

In [6]:
import re
import nltk

nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bhimr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bhimr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

corpus = []
for i in range(len(messages)) :
    review = re.sub('[^a-zA-Z]',' ',messages['message'][i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize (word) for word in review]
    review = ' '.join(review)
    corpus.append(review)

In [8]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [9]:
words = []
for sent in corpus:
    sent_token = sent_tokenize(sent)
    for sent in sent_token :
        words.append(simple_preprocess(sent))

In [10]:
## Word2Vec from Scratch
model = Word2Vec(words)

In [11]:
model.wv.similar_by_word('good')

[('not', 0.9986928105354309),
 ('wa', 0.9986914992332458),
 ('all', 0.9985821843147278),
 ('well', 0.9985135793685913),
 ('oh', 0.9984511733055115),
 ('morning', 0.9984365105628967),
 ('got', 0.9984320402145386),
 ('time', 0.9983425140380859),
 ('about', 0.9983370304107666),
 ('there', 0.9982661604881287)]

In [12]:
def avg_word2vec(doc):
    # remove out-of-vocabulary words
    #sent = [word for word in doc if word in model.wv.index_to_key]
    #print(sent)
    
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)
                #or [np.zeros(len(model.wv.index_to_key))], axis=0)

In [13]:
from tqdm import tqdm
import numpy as np

X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

  0%|                                                                                         | 0/5569 [00:00<?, ?it/s]C:\Users\bhimr\Documents\Downloads\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\bhimr\Documents\Downloads\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|████████████████████████████████████████████████████████████████████████████| 5569/5569 [00:01<00:00, 5043.03it/s]


In [14]:
# Dependent features 
y = messages[list(map(lambda x:len(x)>0 ,corpus))]
y = pd.get_dummies(y['labels'],drop_first=True)
y = y.astype('float32')

In [15]:
print(len(X))
print(y.shape)

5569
(5569, 1)


In [16]:
df = pd.DataFrame()
for i in range(len(X)) :
    df = df.append(pd.DataFrame(X[i].reshape(1,-1)),ignore_index=True)

C:\Users\bhimr\AppData\Local\Temp\ipykernel_27748\63426015.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(X[i].reshape(1,-1)),ignore_index=True)


In [17]:
df.shape

(5569, 100)

In [18]:
df['Output'] = y

In [19]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,Output
0,-0.186956,0.237578,0.119301,0.075623,0.092825,-0.495398,0.175693,0.468579,-0.246880,-0.133524,...,0.150662,0.062244,0.051196,0.444362,0.190794,0.146617,-0.215745,0.167596,-0.007079,0.0
1,-0.172702,0.207465,0.101965,0.064516,0.086489,-0.427414,0.140124,0.407996,-0.214143,-0.108493,...,0.121746,0.049924,0.039331,0.374564,0.160607,0.128120,-0.194452,0.149478,-0.013103,0.0
2,-0.199159,0.252508,0.124136,0.087648,0.072516,-0.536916,0.178278,0.460078,-0.251407,-0.161145,...,0.150043,0.054933,0.028284,0.459354,0.179657,0.097810,-0.247273,0.193170,0.016811,1.0
3,-0.262171,0.325565,0.158004,0.106384,0.129682,-0.671334,0.231864,0.642427,-0.341884,-0.173064,...,0.199389,0.082792,0.076316,0.594923,0.264266,0.216909,-0.298615,0.229187,-0.019140,0.0
4,-0.227112,0.268463,0.142726,0.088338,0.117224,-0.566125,0.193989,0.550051,-0.292174,-0.154824,...,0.165320,0.071333,0.066759,0.506085,0.225476,0.176506,-0.262991,0.189834,-0.017289,0.0


In [20]:
df.dropna(inplace=True)

In [21]:
X = df.drop(columns=['Output'])

In [22]:
X.isnull().sum().values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [23]:
y= df['Output']

In [24]:
y= df['Output']

In [25]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [26]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
3856,-0.250772,0.319639,0.163565,0.107229,0.116987,-0.673390,0.232104,0.626113,-0.332959,-0.194690,...,0.490371,0.201157,0.077057,0.060979,0.592599,0.248994,0.176640,-0.299021,0.229157,-0.003764
2403,-0.185641,0.208131,0.111325,0.071490,0.083698,-0.447684,0.146979,0.414310,-0.224011,-0.132016,...,0.329254,0.126304,0.044048,0.037659,0.384690,0.168608,0.110972,-0.210361,0.150669,0.002831
4447,-0.278622,0.324312,0.174627,0.092176,0.147688,-0.662666,0.231152,0.673344,-0.348218,-0.192132,...,0.526232,0.189563,0.080734,0.083433,0.590026,0.282671,0.231449,-0.306287,0.196729,-0.031489
4786,-0.257627,0.302051,0.157046,0.097929,0.125512,-0.646904,0.218607,0.606509,-0.319965,-0.177175,...,0.473988,0.188158,0.076508,0.059228,0.569666,0.244418,0.172428,-0.294886,0.227651,0.000054
3712,-0.251309,0.305566,0.153987,0.097054,0.134091,-0.632033,0.214304,0.618526,-0.327119,-0.164577,...,0.486174,0.186671,0.072904,0.068945,0.566700,0.254742,0.202819,-0.285296,0.207301,-0.022761


In [27]:
y_train.head()

3856    1.0
2403    0.0
4447    0.0
4786    1.0
3712    0.0
Name: Output, dtype: float32

# GradientBoostingClassifier model 

In [28]:
from sklearn.ensemble import GradientBoostingClassifier

max_depths = [3, 5, 7]
n_estimators = [3, 5, 7, 11]
learning_rates = [0.01, 0.03, 0.05, 0.001]

# Loop over max_depth, n_estimators, and learning_rate
for depth in max_depths:
    for n_est in n_estimators:
        for lr in learning_rates:
            classifier = GradientBoostingClassifier(max_depth=depth, n_estimators=n_est, learning_rate=lr)
            classifier.fit(X_train, y_train)

In [29]:
y_pred = classifier.predict(X_test)

In [30]:
from sklearn.metrics import accuracy_score,classification_report
print(accuracy_score(y_test,y_pred))

0.8613861386138614


In [31]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.86      1.00      0.93       957
         1.0       0.00      0.00      0.00       154

    accuracy                           0.86      1111
   macro avg       0.43      0.50      0.46      1111
weighted avg       0.74      0.86      0.80      1111



C:\Users\bhimr\Documents\Downloads\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\bhimr\Documents\Downloads\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\bhimr\Documents\Downloads\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
!pip install xgboost

In [33]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [34]:
# Define models ( you can add no of option algo in between {    }    
models = {
    "Decision Tree": DecisionTreeClassifier(),
    "logistic Classifier": LogisticRegression(),
    "Random Forest": RandomForestClassifier(),
    "Adaboost":AdaBoostClassifier(),
    "XGBoost": XGBClassifier()
}

# Iterate over models
for name, model in models.items():
    # Fit the model
    model.fit(X_train, y_train)

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Training set performance
    model_train_accuracy = accuracy_score(y_train, y_train_pred)
    model_train_f1 = f1_score(y_train, y_train_pred, average='weighted')
    model_train_precision = precision_score(y_train, y_train_pred)
    model_train_recall = recall_score(y_train, y_train_pred)
    model_train_rocauc_score = roc_auc_score(y_train, y_train_pred)

    # Testing set performance 
    model_test_accuracy = accuracy_score(y_test, y_test_pred)
    model_test_f1 = f1_score(y_test, y_test_pred, average='weighted')
    model_test_precision = precision_score(y_test, y_test_pred)
    model_test_recall = recall_score(y_test, y_test_pred)
    model_test_rocauc_score = roc_auc_score(y_test, y_test_pred)

    # Print model name
    print(f"Model: {name}")

    # Print evaluation metrics for training set
    print("Training Set Performance:")
    print("Accuracy:", model_train_accuracy)
    print("F1 Score:", model_train_f1)
    print("Precision:", model_train_precision)
    print("Recall:", model_train_recall)
    print("ROC AUC Score:", model_train_rocauc_score)
    print("==============================================================")

    # Print evaluation metrics for testing set
    print("\nTesting Set Performance:")
    print("Accuracy:", model_test_accuracy)
    print("F1 Score:", model_test_f1)
    print("Precision:", model_test_precision)
    print("Recall:", model_test_recall)
    print("ROC AUC Score:", model_test_rocauc_score)
    print("==============================================================\n")

Model: Decision Tree
Training Set Performance:
Accuracy: 0.9792932703128516
F1 Score: 0.97868558157277
Precision: 0.9845261121856866
Recall: 0.8583473861720068
ROC AUC Score: 0.9281347320470423

Testing Set Performance:
Accuracy: 0.7848784878487849
F1 Score: 0.7827690193689174
Precision: 0.2108843537414966
Recall: 0.2012987012987013
ROC AUC Score: 0.54004329004329



C:\Users\bhimr\Documents\Downloads\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: logistic Classifier
Training Set Performance:
Accuracy: 0.8663065496286293
F1 Score: 0.8044573603642603
Precision: 0.0
Recall: 0.0
ROC AUC Score: 0.49987012987012985

Testing Set Performance:
Accuracy: 0.8613861386138614
F1 Score: 0.7972403623341058
Precision: 0.0
Recall: 0.0
ROC AUC Score: 0.5

Model: Random Forest
Training Set Performance:
Accuracy: 0.9792932703128516
F1 Score: 0.978920713644498
Precision: 0.9596330275229358
Recall: 0.8819561551433389
ROC AUC Score: 0.9381209347145266

Testing Set Performance:
Accuracy: 0.8514851485148515
F1 Score: 0.811447270616688
Precision: 0.358974358974359
Recall: 0.09090909090909091
ROC AUC Score: 0.5323928944618599

Model: Adaboost
Training Set Performance:
Accuracy: 0.8676569885212694
F1 Score: 0.820850906818372
Precision: 0.5324675324675324
Recall: 0.06913996627318718
ROC AUC Score: 0.5298946584612689

Testing Set Performance:
Accuracy: 0.8586858685868587
F1 Score: 0.8082041491499247
Precision: 0.42105263157894735
Recall: 0.0519480519

# Hyperparameter Tunning

In [35]:
rf_params = { "max_depth":[5, 7,15,10, None],
             "max_features":[5,7,"auto",8],
             "min_samples_split":[2,8,15,20],
             "n_estimators":[100,200,500,1000]}


Xgb_params = {
    "learning_rate":[0.1,0.01],
    "max_depth":[5,8,12,20,30],
    "n_stimators":[100,200,300],
    "colsample_bytree":[0.3,0.4,0.5,0.8,1]
}

In [36]:
rf_params

{'max_depth': [5, 7, 15, 10, None],
 'max_features': [5, 7, 'auto', 8],
 'min_samples_split': [2, 8, 15, 20],
 'n_estimators': [100, 200, 500, 1000]}

In [37]:
Xgb_params

{'learning_rate': [0.1, 0.01],
 'max_depth': [5, 8, 12, 20, 30],
 'n_stimators': [100, 200, 300],
 'colsample_bytree': [0.3, 0.4, 0.5, 0.8, 1]}

In [38]:
randomcv_models = [
                ("RF",RandomForestClassifier(), rf_params),
    ("Xgb Boost",XGBClassifier(), Xgb_params)
    
]

In [39]:
randomcv_models

[('RF',
  RandomForestClassifier(),
  {'max_depth': [5, 7, 15, 10, None],
   'max_features': [5, 7, 'auto', 8],
   'min_samples_split': [2, 8, 15, 20],
   'n_estimators': [100, 200, 500, 1000]}),
 ('Xgb Boost',
  XGBClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bynode=None,
                colsample_bytree=None, device=None, early_stopping_rounds=None,
                enable_categorical=False, eval_metric=None, feature_types=None,
                gamma=None, grow_policy=None, importance_type=None,
                interaction_constraints=None, learning_rate=None, max_bin=None,
                max_cat_threshold=None, max_cat_to_onehot=None,
                max_delta_step=None, max_depth=None, max_leaves=None,
                min_child_weight=None, missing=nan, monotone_constraints=None,
                multi_strategy=None, n_estimators=None, n_jobs=None,
                num_parallel_tree=None, random_state=None, ...),
  {'lear

In [40]:
from sklearn.model_selection import RandomizedSearchCV
model_param = {}
for name, model, params in randomcv_models:
    random =RandomizedSearchCV(estimator=model,
                               param_distributions=params,
                               n_iter=100,
                               cv=3,
                               verbose=2,
                               n_jobs=-1)

    random.fit(X_train, y_train)
    model_param[name] = random.best_params_

for model_name in model_param:
    print(f"--------------------Best Param for {model_name}---------------------------")
    print(model_param[model_name]) 

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits


C:\Users\bhimr\Documents\Downloads\Lib\site-packages\xgboost\core.py:160: UserWarning: [22:17:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_stimators" } are not used.

  warnings.warn(smsg, UserWarning)


--------------------Best Param for RF---------------------------
{'n_estimators': 500, 'min_samples_split': 20, 'max_features': 5, 'max_depth': 5}
--------------------Best Param for Xgb Boost---------------------------
{'n_stimators': 100, 'max_depth': 8, 'learning_rate': 0.01, 'colsample_bytree': 0.5}


In [41]:
# Define models
models = {
    
    "Random Forest": RandomForestClassifier(n_estimators=200,min_samples_split=2,max_features=8,
                                           max_depth= None),
    "Xgb Boost ": XGBClassifier(n_stimators= 200, max_depth= 30, learning_rate= 0.1, colsample_bytree= 1)
}

# Iterate over models
for name, model in models.items():
    # Fit the model
    model.fit(X_train, y_train)

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Training set performance
    model_train_accuracy = accuracy_score(y_train, y_train_pred)
    model_train_f1 = f1_score(y_train, y_train_pred, average='weighted')
    model_train_precision = precision_score(y_train, y_train_pred)
    model_train_recall = recall_score(y_train, y_train_pred)
    model_train_rocauc_score = roc_auc_score(y_train, y_train_pred)

    # Testing set performance 
    model_test_accuracy = accuracy_score(y_test, y_test_pred)
    model_test_f1 = f1_score(y_test, y_test_pred, average='weighted')
    model_test_precision = precision_score(y_test, y_test_pred)
    model_test_recall = recall_score(y_test, y_test_pred)
    model_test_rocauc_score = roc_auc_score(y_test, y_test_pred)

    # Print model name
    print(f"Model: {name}")

    # Print evaluation metrics for training set
    print("Training Set Performance:")
    print("Accuracy:", model_train_accuracy)
    print("F1 Score:", model_train_f1)
    print("Precision:", model_train_precision)
    print("Recall:", model_train_recall)
    print("ROC AUC Score:", model_train_rocauc_score)
    print("==============================================================")

    # Print evaluation metrics for testing set
    print("\nTesting Set Performance:")
    print("Accuracy:", model_test_accuracy)
    print("F1 Score:", model_test_f1)
    print("Precision:", model_test_precision)
    print("Recall:", model_test_recall)
    print("ROC AUC Score:", model_test_rocauc_score)
    print("==============================================================\n")

Model: Random Forest
Training Set Performance:
Accuracy: 0.9792932703128516
F1 Score: 0.9788879626578426
Precision: 0.9630314232902033
Recall: 0.8785834738617201
ROC AUC Score: 0.9366943343334574

Testing Set Performance:
Accuracy: 0.8505850585058505
F1 Score: 0.8097182932578971
Precision: 0.34210526315789475
Recall: 0.08441558441558442
ROC AUC Score: 0.5291461412151067



C:\Users\bhimr\Documents\Downloads\Lib\site-packages\xgboost\core.py:160: UserWarning: [22:18:12] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_stimators" } are not used.

  warnings.warn(smsg, UserWarning)


Model: Xgb Boost 
Training Set Performance:
Accuracy: 0.9792932703128516
F1 Score: 0.9787541805857357
Precision: 0.9771428571428571
Recall: 0.8650927487352446
ROC AUC Score: 0.9309879328091808

Testing Set Performance:
Accuracy: 0.8505850585058505
F1 Score: 0.8131410967183675
Precision: 0.36363636363636365
Recall: 0.1038961038961039
ROC AUC Score: 0.5373190028362442

